In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import scipy.stats as stats

# Question a

In [4]:
y_var = "lwage"
x_vars = ["exp", "wks", "bluecol", "ind", "south", "smsa", "married", "gender", "union", "edu", "colour"]

def between_regression(y, X , N=595, T=7):
    I_T = np.ones((T, 1))
    I_N = np.eye(N)
    D = np.kron(I_N, I_T)
    P_D = D @ np.linalg.inv(D.T @ D) @ D.T
    
    # Averaged y and X per individual
    y_bar = (P_D @ y).reshape(N, T).mean(axis=1, keepdims=True)
    X_bar = (P_D @ X).reshape(N, T, X.shape[1]).mean(axis=1)  # Compute mean for each variable
    
    XtX_inv = np.linalg.inv(X_bar.T @ X_bar)
    beta_B = XtX_inv @ (X_bar.T @ y_bar)
    
    # Compute residuals correctly per individual
    a_i = y_bar - X_bar @ beta_B
    sigma2_alpha = np.sum(a_i**2) / (N - X.shape[1])
    
    # Compute standard errors
    s2_B = sigma2_alpha
    Sigma_B = XtX_inv
    se_B = np.sqrt(s2_B * np.diag(Sigma_B))

    # Compute t-values
    t_values = beta_B.flatten() / se_B
    
    # Compute R squared
    ss_total = np.sum((y_bar - np.mean(y_bar)) ** 2)
    ss_residual = np.sum(a_i**2)
    R2 = 1 - (ss_residual / ss_total)
    
    return beta_B, t_values, se_B, R2, sigma2_alpha

# Load data
file_path = "data/data_assignment1.csv"
data = pd.read_csv(file_path)
data = data.iloc[:, 2:]  # Ignore first two columns

# Prepare y and X
y = data[y_var].to_numpy().reshape(-1, 1)  # Ensure y is (NT, 1)
X = np.column_stack((np.ones(len(y)), data[x_vars].to_numpy()))  # Add constant for intercept

# Run between regression
beta_B, t_values, se_B, R2, sigma2_alpha = between_regression(y, X)
    
# Summary Table
results = pd.DataFrame({
    "Coefficient": beta_B.flatten(),
    "Std Error": se_B,
    "t-Statistic": t_values
}, index=["Intercept"] + x_vars)

print(results)
print(f"R-squared = {R2:.4f}")
print(f"Estimate of variance σ2_α = {sigma2_alpha:.4f}")
    
latex_output = results.style.to_latex()
    
# Output Latex table
with open("between_estimation_results.tex", "w") as f:
    f.write("\\begin{table}[h]\\centering\n")
    f.write(latex_output + "\n")
    f.write("\\caption{Between Estimation Results}\n")
    f.write("\\label{tab:between_estimation}\n")
    f.write("\\end{table}\n")


           Coefficient  Std Error  t-Statistic
Intercept     5.263354   0.207372    25.381238
exp           0.006818   0.001121     6.083924
wks           0.010138   0.003686     2.750502
bluecol      -0.175745   0.034588    -5.081148
ind           0.063580   0.026127     2.433442
south        -0.054965   0.026583    -2.067643
smsa          0.170499   0.026351     6.470270
married       0.134801   0.048685     2.768811
gender       -0.300047   0.055936    -5.364162
union         0.118502   0.029874     3.966651
edu           0.051728   0.005687     9.096532
colour       -0.157205   0.046084    -3.411297
R-squared = 0.5215
Estimate of variance σ2_α = 0.0758


# Question b

In [5]:
y_var = "lwage"
x_vars = ["exp", "wks", "bluecol", "ind", "south", "smsa", "married", "union"]

###########################################################


def within_regression(y, X , N=595, T=7):
    I_T = np.ones((T, 1)) 
    I_N = np.eye(N) 
    I_NT = np.eye(N*T)
    D = np.kron(I_N, I_T)  
    M_D = I_NT - D @ np.linalg.inv(D.T @ D) @ D.T
    
    # Differenced y and X per individual
    y_tilde = M_D @ y
    X_tilde = M_D @ X
    
    XtX_inv = np.linalg.inv(X_tilde.T @ X_tilde)
    beta_W = XtX_inv @ (X_tilde.T @ y_tilde)
    
    # Compute residuals
    u_i = y_tilde - X_tilde @ beta_W
    sigma2_eta = (1 / (N*(T-1)  - X.shape[1])) * np.sum(u_i**2)
    
    # Compute standard errors
    s2_W = sigma2_eta
    Sigma_W = XtX_inv
    se_W = np.sqrt(s2_W * np.diag(Sigma_W))

    # Compute t-values
    t_values = beta_W.flatten() / se_W
    
    #Compute R squared
    ss_total = np.sum((y_tilde - np.mean(y_tilde)) ** 2)
    ss_residual = np.sum((y_tilde - X_tilde @ beta_W) ** 2)
    R2 = 1 - (ss_residual / ss_total)

    return beta_W, t_values, se_W, R2, sigma2_eta, Sigma_W

y = data[y_var].to_numpy()
X = data[x_vars].to_numpy()

beta_W, t_values, se_W, R2, sigma2_eta, Sigma_W = within_regression(y,X)
    
# Summary Table
results = pd.DataFrame({
"Coefficient": beta_W,
"Std Error": se_W,
"t-Statistic": t_values
}, index=x_vars)

print(results)
print(f"R-squared = {R2:.4f}")
print(f"Estimated between regression variance = {sigma2_eta:.4f}")
    
latex_output = results.style.to_latex()
    
# Output Latex table
with open("within_estimation_results.tex", "w") as f:
    f.write("\\begin{table}[h]\\centering\n")
    f.write(latex_output + "\n")
    f.write("\\caption{Within Estimation Results}\n")
    f.write("\\label{tab:within_estimation}\n")
    f.write("\\end{table}\n")

         Coefficient  Std Error  t-Statistic
exp         0.096577   0.001191    81.099181
wks         0.001142   0.000603     1.893728
bluecol    -0.024864   0.013888    -1.790356
ind         0.020757   0.015570     1.333145
south      -0.003198   0.034576    -0.092491
smsa       -0.043727   0.019584    -2.232743
married    -0.030260   0.019137    -1.581241
union       0.034158   0.015042     2.270828
R-squared = 0.6525
Estimated between regression variance = 0.0235


# Question c

In [6]:
data.insert(0, "intercept", 1)
data['individual'] = (data.index // 7) + 1

In [7]:
y_var = "lwage"
x_vars = ["intercept", "exp", "wks", "bluecol", "ind", "south", "smsa", "married", "gender", "union", "edu", "colour"]

# calculate the mean of each individual
df_mean = data.groupby("individual").mean().reset_index()

N=595
T=7
I_T = np.ones((T, 1))

# Calculate theta_hat:
theta2_hat = sigma2_eta / (T * sigma2_alpha + sigma2_eta)
theta_hat = np.sqrt(theta2_hat)

# Change df_mean to numpy array
df_mean_X = df_mean[x_vars].to_numpy()
df_mean_y = df_mean[y_var].to_numpy()

# Calculate the FGLS estimator
first_sum = 0
second_sum = 0

y_star_list = []
X_star_list = []

for i in range(N):
    individual_X = data[data["individual"] == i+1][x_vars]
    individual_X = individual_X.to_numpy()
    Xi_star = 1/sigma2_eta * (individual_X - (1 - theta_hat) * df_mean_X[i])
    first_sum += Xi_star.T @ Xi_star

    individual_y = data[data["individual"] == i+1][y_var].to_numpy()
    yi_star = 1/sigma2_eta * (individual_y - (1 - theta_hat) * df_mean_y[i])
    second_sum += Xi_star.T @ yi_star

    y_star_list.append(yi_star)
    X_star_list.append(Xi_star)

# Calculate the FGLS estimator
beta_FGLS = np.linalg.inv(first_sum) @ second_sum
y_star = np.concatenate(y_star_list)
X_star = np.concatenate(X_star_list)

residuals = y_star - X_star @ beta_FGLS  # Residuals from FGLS

# Compute estimated residual variance
N, k = X_star.shape  # NT = number of observations, k = number of parameters
sigma2_hat = (residuals.T @ residuals) / (N - k)

# Compute standard errors
Sigma_FGLS = np.linalg.inv(first_sum)
se_FGLS = np.sqrt(np.diag(sigma2_hat * Sigma_FGLS))

# Compute t-values
t_values = beta_FGLS.flatten() / se_FGLS

# Compute R squared

R2 = r2_score(y_star, X_star @ beta_FGLS)

# Summary Table
results = pd.DataFrame({
"Coefficient": beta_FGLS,
"Std Error": se_FGLS,
"t-Statistic": t_values
}, index=x_vars)

print(results)
print(f"R-squared = {R2:.4f}")
print(f"Estimated theta = {np.sqrt(theta2_hat):.4f}")

           Coefficient  Std Error  t-Statistic
intercept     4.427204   0.099498    44.495552
exp           0.049531   0.001064    46.530142
wks           0.001621   0.000780     2.078656
bluecol      -0.055488   0.016877    -3.287758
ind           0.007128   0.017587     0.405291
south        -0.012423   0.027399    -0.453419
smsa         -0.024358   0.020439    -1.191742
married      -0.073246   0.023331    -3.139445
gender       -0.329828   0.053454    -6.170263
union         0.067483   0.017364     3.886434
edu           0.102985   0.005990    17.192411
colour       -0.217087   0.060703    -3.576238
R-squared = 0.3739
Estimated theta = 0.2059


## Question D

In [8]:
# We can calculate Hausman statistic only using variables that are not constant over time
x_var_not_constant = ["exp", "wks", "bluecol", "ind", "south", "smsa", "married", "union"]

# Use variable names as index for FGLS results
beta_FGLS_pd = pd.Series(beta_FGLS.flatten(), index=["intercept", "exp", "wks", "bluecol", "ind", "south", "smsa", "married", "gender", "union", "edu", "colour"])
Sigma_FGLS = pd.DataFrame(Sigma_FGLS, index=beta_FGLS_pd.index, columns=beta_FGLS_pd.index)

# For FGLS results keep only values that are calculated for variables that are not constant over time
beta_FGLS_matched = beta_FGLS_pd.loc[x_var_not_constant]
Sigma_FGLS_matched = Sigma_FGLS.loc[x_var_not_constant, x_var_not_constant]

# Compute Hausman test statistic
beta_diff = beta_FGLS_matched - beta_W
var_diff = Sigma_W - Sigma_FGLS_matched
statistic = beta_diff.T @ np.linalg.inv(var_diff) @ beta_diff
p_value = 1 - stats.chi2.cdf(statistic, len(x_var_not_constant))

print(f"Hausman Test Statistic: {statistic:.4f}")
print(f"p-value: {p_value:.4f}")



Hausman Test Statistic: 36.9639
p-value: 0.0000


In [10]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load dataset
df = pd.read_csv("data/data_assignment1.csv")

# Define dependent and independent variables 
y_var = "lwage"
x_vars_corrected = ["wks", "bluecol", "ind", "south", "smsa", "married", "union"]  

# Add ID and Year columns based on dataset structure
df["ID"] = np.repeat(np.arange(1, 596), 7)  
df["Year"] = np.tile(np.arange(1976, 1983), 595) 

# Compute means for demeaning
y_mean_i = df.groupby("ID")[y_var].transform("mean")  # Individual mean
y_mean_t = df.groupby("Year")[y_var].transform("mean")  # Time mean
y_mean_overall = df[y_var].mean()  # Overall mean

# Compute transformed y
df["y_twfe"] = df[y_var] - y_mean_i - y_mean_t + y_mean_overall

# Compute transformed X variables (demeaning)
for var in x_vars_corrected:
    x_mean_i = df.groupby("ID")[var].transform("mean")  # Individual mean
    x_mean_t = df.groupby("Year")[var].transform("mean")  # Time mean
    x_mean_overall = df[var].mean()  # Overall mean
    df[f"x_twfe_{var}"] = df[var] - x_mean_i - x_mean_t + x_mean_overall

# Prepare X matrix 
X_twfe = df[[f"x_twfe_{var}" for var in x_vars_corrected]]  

# Run OLS 
y_twfe = df["y_twfe"]
model_twfe_final = sm.OLS(y_twfe, X_twfe).fit()

# Print final regression results
print(model_twfe_final.summary())


                                 OLS Regression Results                                
Dep. Variable:                 y_twfe   R-squared (uncentered):                   0.005
Model:                            OLS   Adj. R-squared (uncentered):              0.003
Method:                 Least Squares   F-statistic:                              3.064
Date:                Wed, 26 Feb 2025   Prob (F-statistic):                     0.00321
Time:                        13:36:02   Log-Likelihood:                          2250.7
No. Observations:                4165   AIC:                                     -4487.
Df Residuals:                    4158   BIC:                                     -4443.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------